In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import numpy as np
tfe.enable_eager_execution()

In [2]:
#기본 값
df_path = './dataset/chat_train_ids.in'
buckets = [(12, 40)]
PAD_ID = 0
GO_ID = 1
EOS_ID = 2
UNK_ID = 3
user_vocab_size = 9687
bot_vocab_size = 12169

## Preprocessing Dataset

- Datset API사용 (큐 기반)
- Clear Structure and Simplicity

In [5]:
#Load Sample Dataset

def read_train_set(train_path, max_size=None):
    train_set = [[] for _ in range(buckets[0][0])]
    
    with open(train_path, "r", encoding="utf-8") as ts:
        counter = 0
        while not max_size or counter < max_size:
            
            # iterator 성질을 이용하여, 유저, 봇 발화 불러오기
            user_tokens, bot_tokens = ts.readline(), ts.readline()
            
            # 유저 발화에서 모르는 토큰이 있으면 언노운 아이디 부여한다
            user_token_ids = [int(x) for x in user_tokens.split()]

            # 모르는 토큰이 있으면 언노운 아이디 부여
            for i, user_token_id in enumerate(user_token_ids):
                if user_token_id > user_vocab_size:
                    user_token_ids[i] = UNK_ID

            # 유저 버킷 크기로 잘라낸다
            user_token_bucket = user_token_ids[:buckets[0][0]]
            user_token_len = len(user_token_bucket)

            # 봇 발화에서 모르는 토큰이 있으면 언노운 아이디 부여한다
            bot_token_ids = [int(x) for x in bot_tokens.split()]
            for i, bot_token_id in enumerate(bot_token_ids):
                if bot_token_id > bot_vocab_size:
                    bot_token_ids[i] = UNK_ID

            # 봇 버킷 크기로 잘라낸다
            bot_token_bucket = bot_token_ids[:(buckets[0][1]-1)]

            # 유저 토큰 길이에 따라 유저 발화와 봇 발화를 모은다
            train_set[user_token_len - 1].append([user_token_bucket, bot_token_bucket])

            counter += 1

            # 진행도를 알아볼 수 있도록 1000번마다 프린트한다
            if counter % 1000 == 0:
                print("read data line %d" % counter)

        return train_set
    
train_set = read_train_set(df_path, 5000)
train_len_hist = [len(train_set[i]) for i in range(len(train_set))]
train_size = float(sum(train_len_hist))

print("sep by train, {}".format(train_len_hist))
print("train_size: {}".format(train_size))

read data line 1000
read data line 2000
read data line 3000
read data line 4000
read data line 5000
sep by train, [0, 0, 20, 12, 35, 91, 180, 236, 229, 144, 76, 3977]
train_size: 5000.0


In [6]:
# train_set[4][0][0] #고객 발화
# train_set[4][0][1] #봇 발화

df_train = [train_set[4][i] for i in range(len(train_set[4]))] #Sample로 input size 가 5인 발화를 추출한다.
df_user = [df_train[i][0] for i in range(len(df_train))]
df_bot = [df_train[i][1] for i in range(len(df_train))]

In [13]:
#skip

embed_size = 1
window_size = 3
dict_size = 5
filter_size = window_size*embed_size

ids = tf.constant(df_train)


def input_fn():
    ids = tf.constant([[5, 6, 10, 9, 2], [5, 6, 10, 60, 2]])
    label_ids = tf.constant([[4, 22, 163, 649, 2], [4, 22, 163, 649, 2]])
    return {'ids': ids,
            'init_label': label_ids[:, :window_size],
            'label_length': int(label_ids.shape[1])-window_size
            }, label_ids[:, window_size:]

ValueError: Can't convert non-rectangular Python sequence to Tensor.

In [14]:
ids = tf.constant([[1, 2, 3, 4, 3, 2, 1], [1, 2, 3, 2, 3, 2, 3]])
label_ids = tf.constant([[0, 0, 0, 1, 3, 1, 3, 2], [0, 0, 0, 2, 1, 3, 4, 4]]) #무조건 앞에 3개를... 
init_label = label_ids[:, :window_size]
label_length = int(label_ids.shape[1]) - window_size
label_ids = label_ids[:, window_size:]

print(init_label)
print(label_length)
print(label_ids)

tf.Tensor(
[[0 0 0]
 [0 0 0]], shape=(2, 3), dtype=int32)
5
tf.Tensor(
[[1 3 1 3 2]
 [2 1 3 4 4]], shape=(2, 5), dtype=int32)


In [15]:
embeddings = tfe.Variable(tf.truncated_normal([dict_size, embed_size]), trainable=False)
print(embeddings.shape)

(5, 1)


In [16]:
class Embedding(tf.layers.Layer):
    """ 임베딩 층"""
    def __init__(self, vocab_size, embedding_dim, **kwargs):
        super(Embedding, self).__init__(**kwargs)
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        
    def build(self, _):
        self.embedding = self.add_variable(
        "embedding_kernel",
        shape=[self.vocab_size, self.embedding_dim],
        dtype=tf.float32,
        initializer = tf.random_uniform_initializer(-0.1, 0.1),
        trainable=True)
    
    def call(self, x):
        return tf.nn.embedding_lookup(self.embedding, x)
    

Embedding(50, 100)

In [17]:
# embedding
input_embeds = tf.nn.embedding_lookup(embeddings, ids)
print(input_embeds)

input_flat = tf.layers.flatten(input_embeds)
input_flat = tf.expand_dims(input_flat, -1)

init_label = tf.nn.embedding_lookup(embeddings, init_label)
init_label = tf.layers.flatten(init_label)
init_label = tf.expand_dims(init_label, -1)

print(input_flat.shape, init_label.shape)

label_onehot = tf.one_hot(label_ids, depth=dict_size, dtype=tf.float32)
print(label_onehot)

tf.Tensor(
[[[ 0.0086463 ]
  [-0.95740545]
  [-1.38855588]
  [-0.24032409]
  [-1.38855588]
  [-0.95740545]
  [ 0.0086463 ]]

 [[ 0.0086463 ]
  [-0.95740545]
  [-1.38855588]
  [-0.95740545]
  [-1.38855588]
  [-0.95740545]
  [-1.38855588]]], shape=(2, 7, 1), dtype=float32)
(2, 7, 1) (2, 3, 1)
tf.Tensor(
[[[ 0.  1.  0.  0.  0.]
  [ 0.  0.  0.  1.  0.]
  [ 0.  1.  0.  0.  0.]
  [ 0.  0.  0.  1.  0.]
  [ 0.  0.  1.  0.  0.]]

 [[ 0.  0.  1.  0.  0.]
  [ 0.  1.  0.  0.  0.]
  [ 0.  0.  0.  1.  0.]
  [ 0.  0.  0.  0.  1.]
  [ 0.  0.  0.  0.  1.]]], shape=(2, 5, 5), dtype=float32)


In [18]:
# encoder
encoder_conv = tf.layers.conv1d(
        inputs=input_flat,
        filters=2*embed_size,
        kernel_size=filter_size,
        strides=embed_size,
        padding='same')

print(encoder_conv.shape)
#embed_size, 반으로 나눈다.
encoder_glu = encoder_conv[:, :, embed_size:]*tf.nn.sigmoid(encoder_conv[:, :, :embed_size])
print(encoder_glu.shape)

(2, 7, 2)
(2, 7, 1)


In [20]:
#decoder: 그림에서 왼쪽에서 하나의 블록을 구하는 식

decoder_conv = tf.layers.conv1d(
        inputs=init_label,
        filters=2*embed_size,
        kernel_size=filter_size,
        strides=embed_size)

decoder_glu = decoder_conv[:, :, embed_size:]*tf.nn.sigmoid(decoder_conv[:, :, :embed_size])

print(decoder_conv.shape)
print(decoder_glu.shape)

tiled_decoder_glu = tf.tile(decoder_glu, [1, int(encoder_glu.shape[1]), 1]) #decoder 단어 1개, encoder는 7개단어)
print(tiled_decoder_glu.shape)

dot_prod = tf.matmul(encoder_glu, decoder_glu, transpose_b=True)
print(dot_prod.shape)

(2, 1, 2)
(2, 1, 1)
(2, 7, 1)
(2, 7, 1)


In [21]:
attention = tf.nn.softmax(dot_prod, axis=1) #수정해야함
#tf.reduce_sum(attention, axis=1)
print(attention.shape)

z_plus_e = encoder_glu + input_embeds
print(z_plus_e.shape)

tiled_attention = tf.tile(attention, [1, 1, embed_size])
print(tiled_attention.shape)

c = tf.reduce_sum(tiled_attention*z_plus_e, axis=1)
print(c.shape)

decoder_glu = tf.reshape(decoder_glu, [-1, embed_size])
print(decoder_glu.shape)

logits = tf.layers.dense(c+decoder_glu, dict_size)
print(logits.shape)

out = tf.nn.softmax(logits)
print(out)

next_id = tf.argmax(out, axis=1)
next_embeds = tf.nn.embedding_lookup(embeddings, next_id)
next_embeds = tf.expand_dims(next_embeds, -1)

(2, 7, 1)
(2, 7, 1)
(2, 7, 1)
(2, 1)
(2, 1)
(2, 5)
tf.Tensor(
[[ 0.2683596   0.21093608  0.1373333   0.22239776  0.16097324]
 [ 0.3248516   0.21198468  0.09905788  0.2328326   0.13127331]], shape=(2, 5), dtype=float32)


In [26]:
decoder_input = tf.concat([init_label[:, embed_size:], next_embeds], axis=1)

next_ids.append(next_id)
outs.append(logits)

NameError: name 'next_ids' is not defined